<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/mistral_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# https://aws.amazon.com/blogs/machine-learning/mistral-7b-foundation-models-from-mistral-ai-are-now-available-in-amazon-sagemaker-jumpstart/
# https://github.com/mistralai/mistral-src

#It looks great
# https://mistral.ai/news/mixtral-of-experts/

# https://www.nytimes.com/2023/12/10/technology/mistral-ai-funding.html

# https://aws.amazon.com/blogs/machine-learning/mistral-7b-foundation-models-from-mistral-ai-are-now-available-in-amazon-sagemaker-jumpstart/

!pip install sagemaker
!pip install boto3
!pip install --upgrade urllib3
!pip install colab-env --upgrade

import colab_env
import boto3
import sagemaker
from sagemaker.jumpstart.model import JumpStartModel
import os

aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
region=os.getenv("AWS_DEFAULT_REGION")
output=os.getenv("AWS_DEFAULT_OUTPUT")

iam_client = boto3.client("iam")

role = iam_client.get_role(
    RoleName=os.getenv("ROLENAME")
)

ROLE_ARN = role['Role']['Arn']

print('huggingface-llm-mistral-7b-instruct - JumpStartModel')
model_version='2.0.0'
model = JumpStartModel(model_id="huggingface-llm-mistral-7b-instruct", model_version='2.0.0', role=ROLE_ARN)
predictor = model.deploy()


In [2]:

def predict_sentiment(prompt,predictor,modelid):
    if modelid == 0:
       INPUT= "<s>[INST]  " + prompt + " [/INST]"
    else:
       INPUT= "%s"%prompt

    payload = {
        #"inputs": "<s>[INST]  " + prompt + " [/INST]",
        "inputs": "%s"%INPUT,
        "parameters": {
            "do_sample": True,
            "top_p": 0.9,
            "temperature": 0.9,
            "max_new_tokens": 512,
            "return_full_text": False,
            #"stop": ["<|endoftext|>", "</s>"]
        },
    }
    return predictor.predict(payload,custom_attributes="accept_eula=true")

In [8]:
prompt0 = "what is the 40% of 30?"
prompt0 = "what is the 20.5% of 40?"
prompt0 = "what is the 30% of 650?"
prompt0 = "As a data scientist, can you explain the concept of regularization in machine learning?"
#prompt0='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
#prompt0='Which country has the most natural lakes? Answer with only the country name.'

print()
print('Prompt #0: %s'%prompt0)
print()
sentiment = predict_sentiment(prompt0,predictor,0)
#print(sentiment)
print('OUTPUT PROMPT0:')
print()
print(f"Answer: {sentiment[0]['generated_text']}")
print()


Prompt #0: As a data scientist, can you explain the concept of regularization in machine learning?

OUTPUT PROMPT0:

Answer:  Regularization is a technique used in machine learning to prevent overfitting of models. Overfitting occurs when a model is too complex and fits the training data too well, including the noise and errors in the data. As a result, the model may not generalize well to new data and may not make accurate predictions. 

Regularization adds a penalty term to the cost function of the model, which encourages the model to have smaller coefficients or weights. This results in a simpler model that is less likely to overfit the data. There are two main types of regularization: L1 and L2 regularization. L1 regularization adds a penalty term equal to the absolute value of the coefficients, while L2 regularization adds a penalty term equal to the square of the coefficients.

In summary, regularization is an important technique in machine learning that helps prevent overfittin

In [9]:
#  Frank Morales created this cell on December 14, 2023; it fully allows automatically the deletion of endpoints, models, and endpoint configurations.

#!pip install colab-env --upgrade
import colab_env
import os

aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region=os.getenv("AWS_DEFAULT_REGION")
aws_output=os.getenv("AWS_DEFAULT_OUTPUT")

#!pip install boto3
#aws_region = 'us-east-1'
import boto3

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

def cleanup_sagemaker_resources(resource_name,resourceid):

    if resourceid==0:
       response=sagemaker_client.list_endpoints()
    elif resourceid==1:
         response=sagemaker_client.list_models()
    elif resourceid==2:
         response=sagemaker_client.list_endpoint_configs()

    print(resource_name)
    #resource_nametmp='%s'%resource_name[0:len(resource_name)-1]
    #print('%sName'%resource_nametmp)

    number_of_endpoints=len(response['%s'%resource_name])
    for i in range(number_of_endpoints):
        resource_nametmp='%s'%resource_name[0:len(resource_name)-1]
        print('%sName'%resource_nametmp)
        print(response['%s'%resource_name][i]['%sName'%resource_nametmp])

        if resourceid==0:
           endpoint_name=response['%s'%resource_name][i]['%sName'%resource_nametmp]
           sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
        elif resourceid==1:
           sagemaker_client.delete_model(ModelName=response['Models'][i]['ModelName'])
        elif resourceid==2:
           sagemaker_client.delete_endpoint_config(EndpointConfigName=response['EndpointConfigs'][i]['EndpointConfigName'])

    print("\n==================================\n")

cleanup_sagemaker_resources('Endpoints',0)

cleanup_sagemaker_resources('Models',1)

cleanup_sagemaker_resources('EndpointConfigs',2)

Endpoints
EndpointName
hf-llm-mistral-7b-instruct-2023-12-21-02-49-35-887


Models
ModelName
hf-llm-mistral-7b-instruct-2023-12-21-02-49-35-885


EndpointConfigs
EndpointConfigName
hf-llm-mistral-7b-instruct-2023-12-21-02-49-35-887


